# StyleGAN2 ADA PyTorch Generator

In [ ]:
#@title 1. Mount your Google Drive
#@markdown Access your Google Drive to load pre-trained models and save the results.

#@markdown

#@markdown
from google.colab import drive
drive.mount('/content/drive')




In [ ]:
#@title 2. Installation of libraries
#@markdown StyleGAN2-ADA will be installed on your Google Drive to speed up the training process.

#@markdown Run this cell. If you have already installed the repository, it will skip the installation process and update the repository directory. If you haven't installed it, it will install all the necessary files. You will notice some errors, ignore them for now as they are compatibility issues with Colab that do not influence our work.
import os
import shlex

!pip install numpy==1.22.4
import numpy as np


if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

# 3. Generation of results

In [ ]:
#@title 3.1 Image Generation
#@markdown Write the directory where the generated images will be saved:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-2" #@param {type: "string"}
#@markdown Path to the pretrained model. The easiest way is to look for the file in the Drive directory found in the left panel, click on the three dots and choose the `copy path` option, then paste it here:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00070-amonites-11gb-gpu-noaug-resumecustom/network-snapshot-000020.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Generation Parameters**

#@markdown "Truncation", let's say, rounds or truncates the values ​​of the latent space. This can have a subtle or dramatic effect on your images depending on the value you use. The lower the number, the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it is infinite.
truncation =   0.8#@param {type: "number"}
#@markdown This allows you to choose random seeds (starting points) from the model. Recall that our input to StyleGAN is a matrix of 512 points in three dimensions. These seeds will generate those 512x512x512 = 134217728 values.

#@markdown Each seed will generate a different random route. But the same seed value will also always generate the same random array, so we can then use it for other purposes, such as interpolation.
#@markdown
#@markdown Provide a number, a comma-separated list of integers, or a range between values ​​using a hyphen - .
seeds = "1200-1250" #@param {type: "string"}
#@markdown Random image generation (when active the seed parameter will be ignored)
gen_random = True #@param {type: "boolean"}
#@markdown Number of random images to generate
n_imgs = 20 #@param {type: "integer"}
#@markdown ---
#@markdown **Generation of non-square images**

#@markdown It is possible to make the model generate images that are not square. This is not as good as training a rectangular model, but with the right model it can still look good.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Width
width = 640 #@param {type: "integer"}
#@markdown Height
height = 480 #@param {type: "integer"}
#@markdown Fill style to apply to the "extra" space.
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = ','.join(str(s) for s in list(set(list(np.random.randint(4294967295, size=n_imgs)))))
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''


outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare}

In [ ]:
#@title 3.2 Image interpolation
#@markdown Write the directory where the generated images will be saved:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-int2" #@param {type: "string"}
#@markdown Path to the pretrained model. The easiest way is to look for the file in the Drive directory found in the left panel, click on the three dots and choose the `copy path` option, then paste it here:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00070-amonites-11gb-gpu-noaug-resumecustom/network-snapshot-000020.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Generation Parameters**

#@markdown "Truncation", let's say, rounds or truncates the values ​​of the latent space. This can have a subtle or dramatic effect on your images depending on the value you use. The lower the number, the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it is infinite.
truncation =  0.8#@param {type: "number"}

#@markdown This allows you to choose random seeds (starting points) from the model. Recall that our input to StyleGAN is a matrix of 512 points in three dimensions. These seeds will generate those 512x512x512 = 134217728 values.
#@markdown
#@markdown Provide a number, a comma-separated list of integers, or a range between values ​​using a hyphen - .
seeds = "632,633,634,634,632" #@param {type: "string"}
#@markdown Random image generation (when active the seed parameter will be ignored)
gen_random = False #@param {type: "boolean"}
#@markdown Number of random images to generate
n_imgs =  5#@param {type: "integer"}
gen_loop = False
#@markdown ---
#@markdown **Interpolation Parameters**
#@markdown
#@markdown Number of frames per image
frames = 120 #@param {type: "integer"}

interpolation = 'linear'
#@markdown Latent Space
space = 'z' #@param ['z', 'w']
#@markdown ---
#@markdown **Generation of non-square images**

#@markdown It is possible to make the model generate images that are not square. This is not as good as training a rectangular model, but with the right model it can still look good.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Width
width = 480 #@param {type: "integer"}
#@markdown Height
height = 720 #@param {type: "integer"}
#@markdown Fill style to apply to the extra space.
scale_type = 'padside' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = list(set(list(np.random.randint(4294967295, size=n_imgs))))
  if gen_loop:
    seeds.append(seeds[-1])
  seeds = ','.join(str(s) for s in seeds)
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare} --space={space} --process="interpolation" --interpolation={interpolation} --frames={frames}

In [ ]:
#@title 3.3 Transversal Truncation

#@markdown You can then take a seed and see the changes in it over any amount of truncation. -1 to 1 will be pretty realistic images, but the further you zoom out, the weirder they get.

#@markdown Write the directory where the generated images will be saved:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-trunc4" #@param {type: "string"}
#@markdown Path to the pretrained model. The easiest way is to look for the file in the Drive directory found in the left panel, click on the three dots and choose the `copy path` option, then paste it here:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00000-coral-bleached-ver-11gb-gpu-noaug-resumeffhq512/network-snapshot-000016.pkl" #@param {type: "string"}

#@markdown Choose only one seed. Choose a favorite from your generated images:
seed = 939915100  #@param {type: "integer"}
#@markdown Initial truncation value:
start =   0#@param {type: "number"}
#@markdown Value where truncation stops. Must be greater than the initial value: (An error will occur if not)
stop = 1.5  #@param {type: "number"}
#@markdown How much each frame should increase the truncation value. Set a very small value if you want a long, slow interpolation. (stop-start/increment = total frames)
increment = 0.005  #@param {type: "number"}

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --process="truncation" --outdir={outdir} --start={start} --stop={stop} --increment={increment} --seeds={seed} --network={network}